# Project : Random Acts of Pizza

Overview:


In [3]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import nltk
import json

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

Print the request text and the outcome for 5 requests from the training set

In [5]:
with open('train.json') as data_file:    
    train_list = json.load(data_file)

train_data = np.array(train_list)

print('Dimensions of train data set ', train_data.shape)
print('There are ',len(train_list), ' observations.\n')
yescount = 0;
nocount = 0;
for i in range(0, len(train_list)):
    if yescount > 3 and nocount > 3:
        break 
    if train_data[i]["requester_received_pizza"] == False and nocount < 4:
        print(i, '. Outcome: ',train_list[i]["requester_received_pizza"],'\n', train_list[i]["request_text"],'\n')
        nocount += 1
    elif train_data[i]["requester_received_pizza"] == True and yescount < 4:
        print(i, '. ',train_list[i]["requester_received_pizza"],'\n', train_list[i]["request_text"],'\n')
        yescount += 1
    else:
        continue

Dimensions of train data set  (4040,)
There are  4040  observations.

0 . Outcome:  False 
 Hi I am in need of food for my 4 children we are a military family that has really hit hard times and we have exahusted all means of help just to be able to feed my family and make it through another night is all i ask i know our blessing is coming so whatever u can find in your heart to give is greatly appreciated 

1 . Outcome:  False 
 I spent the last money I had on gas today. Im broke until next Thursday :( 

2 . Outcome:  False 
 My girlfriend decided it would be a good idea to get off at Perth bus station when she was coming to visit me and has since had to spend all her money on a taxi to get to me here in Dundee. Any chance some kind soul would get us some pizza since we don't have any cash anymore? 

3 . Outcome:  False 
 It's cold, I'n hungry, and to be completely honest I'm broke. My mum said we're having leftovers for dinner. A random pizza arriving would be nice.

Edit: We had left